In [1]:
import pandas as pd
from src import Utils, LSTM

/tmp/ipykernel_48777/519450756.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
/home/nate/miniconda3/lib/python3.9/site-packages/spacy/util.py:918: UserWarning: [W094] Model 'tl_calamancy_md' (0.1.0) specifies an under-constrained spaCy version requirement: >=3.5.0. This can lead to compatibility problems with older versions, or as new spaCy versions are released, because the model may say it's compatible when it's not. Consider changing the "spacy_version" in your meta.json to a version range, with a lower and upper pin. For example: >=3.7.4,<3.8.0
  warnings.warn(warn_msg)
/home/nate/minicond

In [2]:
MODEL_FOLDER = 'model_lstm'

In [3]:
dataset = Utils.read_csv_file('datasets/datasetall.csv')
dataset

CSV file read successfully!


,text,label
0,Binay: Patuloy ang kahirapan dahil sa maling p...,0
1,SA GOBYERNONG TAPAT WELCOME SA BAGUO ANG LAHAT...,0
2,wait so ur telling me Let Leni Lead mo pero NY...,1
3,[USERNAME]wish this is just a nightmare that ...,0
4,doc willie ong and isko sabunutan po,0
...,...,...
28456,"Bisaya, Probinsyano/a, mostly Bisaya = katulong",1
28457,Amnesia. In my whole life wala pa ako nakasala...,1
28458,Kontrabida na ilang beses na tinalo at obvious...,1
28459,Yung antagonist laging kailangang sobrang sama...,1


In [4]:
Utils.seed_random_number_generators()

Random number generators seeded.


In [5]:
TEST_SIZE = 0.2

X_train, X_test, y_train, y_test = Utils.get_train_test_split(dataset, TEST_SIZE)

In [6]:
X_train

0        Yan, ganyan kagarapalan. Kung tatangapin mo ya...
1                                        Exactly!Elections
2                               sabi ni onej Let Leni Lead
3                 kahit saan mag scroll Mar Roxas lagi zzz
4        Lady Chismosa [USERNAME] [USERNAME] and [USERN...
                               ...                        
19919    Binaylibre gamot? Libre pahospital? Tapos no i...
19920    F*CK You VP Binay! That commercial is the most...
19921     marcos magnanakaw marcos not a hero No To Cor...
19922    "[USERNAME]: Stop meddling with INC  Binay tel...
19923                                Uunlad tayo kay Binay
Name: text, Length: 19924, dtype: object

In [7]:
y_train

0        1
1        0
2        0
3        1
4        1
        ..
19919    0
19920    1
19921    1
19922    1
19923    0
Name: label, Length: 19924, dtype: int64

In [8]:
X_test

0       Bawal si Poe kasi hindi natural born? Pero pwe...
1            +. road to + Angat Buhay Lahat Let Leni Lead
2       juju [USERNAME] Let Leni Lead < jake ejercito ...
3              I hope you choke on your bigotry[USERNAME]
4       cie [USERNAME] No To Marcos Returnto Power Nev...
                              ...                        
8532    Really? You're tryna show me this shit? Wow! I...
8533    Grabe Marnakikiparehas nala't patalastas ni Bi...
8534    Jose Montemayor Jr. elitist piece of sht CNNPH...
8535    Here goes Binay with his annoying hits ?????? ...
8536    What's with the caption?????Senyales ng Disban...
Name: text, Length: 8537, dtype: object

In [9]:
y_test

0       1
1       0
2       0
3       1
4       1
       ..
8532    1
8533    1
8534    1
8535    1
8536    0
Name: label, Length: 8537, dtype: int64

In [10]:
epochs = [100, 200, 300]
learning_rate = [0.1, 0.01, 0.001, 0.0001, 0.00001]
batch_size = [32, 64, 128]
hidden_size = [100, 200, 300, 400]
num_layers = [1, 2, 3]

train_lstm = LSTM.LstmPipeline

In [11]:
train_lstm.set_params(
  # lstm__train_split=None,
  lstm__module__hidden_size=400,
  lstm__optimizer__lr=0.015,
  lstm__optimizer__weight_decay=0.00001,
  lstm__max_epochs=20,
  lstm__batch_size=32,
)

Pipeline(steps=[('tokenizer', CalamancyTokenizer()),
                ('lstm',
                 <class 'skorch.classifier.NeuralNetClassifier'>[uninitialized](
  module=<class 'src.LSTM.LstmModel'>,
  module__hidden_size=400,
))])

In [12]:
train_lstm

Pipeline(steps=[('tokenizer', CalamancyTokenizer()),
                ('lstm',
                 <class 'skorch.classifier.NeuralNetClassifier'>[uninitialized](
  module=<class 'src.LSTM.LstmModel'>,
  module__hidden_size=400,
))])

In [13]:
train_lstm.fit(X_train, y_train)

  0%|          | 0/624 [00:00<?, ?it/s]

  epoch    train_loss    valid_acc    valid_loss    cp     dur
-------  ------------  -----------  ------------  ----  ------
      1        0.7374       0.4991        0.7136     +  9.5255


  0%|          | 0/624 [00:00<?, ?it/s]

      2        0.7055       0.4991        0.6952     +  9.1981


  0%|          | 0/624 [00:00<?, ?it/s]

...

  0%|          | 0/624 [00:00<?, ?it/s]

     99        0.4830       0.7749        0.5251        8.3896


  0%|          | 0/624 [00:00<?, ?it/s]

    100        0.4687       0.7822        0.5031        8.3666


Pipeline(steps=[('tokenizer', CalamancyTokenizer()),
                ('lstm',
                 <class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=LstmModel(
    (lstm): LSTM(200, 400, num_layers=2, batch_first=True)
    (fc): Linear(in_features=400, out_features=2, bias=True)
  ),
))])

In [14]:
accuracy, recall, precision, f1 = Utils.get_prediction_results(
  X_test,
  y_test,
  train_lstm,
)

Accuracy: 0.7920815274686658
Recall: 0.8652103183825238
Precision: 0.7570150467669784
F1-score: 0.8075046090445721


In [15]:
Utils.save_trained_model(train_lstm, f"{MODEL_FOLDER}/LSTM")

Ensemble model saved to Pipeline(steps=[('tokenizer', CalamancyTokenizer()),
                ('lstm',
                 <class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=LstmModel(
    (lstm): LSTM(200, 400, num_layers=2, batch_first=True)
    (fc): Linear(in_features=400, out_features=2, bias=True)
  ),
))]).pkl


In [16]:
data_frame = pd.DataFrame(
    train_lstm['lstm'].history
).set_index('epoch')
data_frame.to_csv(f'{MODEL_FOLDER}/lstm_history.csv')

In [17]:
metrics_data_frame = pd.DataFrame([{
  'accuracy': accuracy,
  'recall': recall,
  'precision': precision,
  'f1': f1,
}])
metrics_data_frame.to_csv(f'{MODEL_FOLDER}/lstm_metrics.csv')